In [1]:
!pip install simpletransformers

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle
from pathlib import Path
import time, os, csv, srsly, collections

In [3]:
file_name_answers = "main_3_per_cluster_download.66081fcf-3ef5-48ea-97e0-49298d29b477"
file_path_answers = "/Users/ash/Desktop/"+file_name_answers+".jsonl"
data = srsly.read_jsonl(file_path_answers)
data1 = srsly.read_jsonl(file_path_answers)
sent_dict_train = []
sent_dict_test = []
count = 1
tot_len = 0

for n in data1:
    if n['answer'] == "accept":
        if n['_session_id'] == "main_3_per_cluster-Kameron":
            tot_len += 1

# 70% training, 30% testing
training_data_index = int(tot_len*0.7)

for entry in data:
    if "text" in entry:
        text = entry["text"]
        
    if entry['answer'] == "accept":
        if entry['_session_id'] == "main_3_per_cluster-Kameron":
            for relation in entry['spans']:
                sent_dict_tmp = {}
                sent_dict_tmp["Sentence #"] = "Sentence " + str(count)
                if ("label" in relation) and ("start" in relation) and ("end" in relation):
                    sent_dict_tmp["text"] = text
                    child_span_start = relation["start"]
                    child_span_end = relation["end"]
                    word = text[child_span_start:child_span_end]
                    if relation["label"] == "base":
                        sent_dict_tmp["word"] = text[child_span_start: child_span_end]
                        sent_dict_tmp["tag"] = relation["label"]
                    else:
                        sent_dict_tmp["word"] = text[child_span_start: child_span_end]
                        sent_dict_tmp["tag"] = "O"
                if training_data_index != 0:
                    sent_dict_train.append(sent_dict_tmp)
                else:
                    sent_dict_test.append(sent_dict_tmp)
            if training_data_index > 0:
                training_data_index -= 1
        count += 1
    

train_data = pd.DataFrame.from_dict(sent_dict_train)
train_data = train_data.fillna(method ="ffill")
train_data["Sentence #"] = LabelEncoder().fit_transform(train_data["Sentence #"] )
train_data.rename(columns={"Sentence #":"sentence_id", "word":"words", "tag":"labels"}, inplace = True)
train_data["labels"] = train_data["labels"].str.upper()

test_data = pd.DataFrame.from_dict(sent_dict_test) 
test_data = test_data.fillna(method ="ffill")
test_data["Sentence #"] = LabelEncoder().fit_transform(test_data["Sentence #"] )
test_data.rename(columns={"Sentence #":"sentence_id", "word":"words", "tag":"labels"}, inplace = True)
test_data["labels"] = test_data["labels"].str.upper()

In [4]:
ans_dict = collections.defaultdict(list)
for index, row in test_data.iterrows():
    if row["labels"] == "BASE":
        ans_dict[row["text"]].append(row["words"])

print(ans_dict)

defaultdict(<class 'list'>, {'Around the wine-growing world, smart producers have contemplated and experimented with adaptations, not only to hotter summers, but also to warmer winters, droughts and the sort of unexpected, sometimes violent events that stem from climate change: freak hailstorms, spring frosts, flooding and forest fires, just to name a few. IMPLIED_BASE IMPLIED_BASE': ['summers', 'winters', 'droughts', 'climate change', 'hailstorms', 'frosts', 'flooding', 'forest fires'], '“Extreme heat, powerful storms and floods, year-round wildfires, droughts, and other climate-related events have already caused thousands of deaths and displaced tens of thousands of people in the U.S. from their homes, with significant personal loss and mental health impacts especially for first responders and children,” the letter warns. IMPLIED_BASE IMPLIED_BASE': ['heat', 'storms', 'floods', 'wildfires', 'droughts', 'deaths', 'people', 'personal loss', 'health'], 'Historical warming has likely inc

In [5]:
print(len(train_data.text.unique()))
print(len(test_data.text.unique()))

214
93


# Model Training


In [6]:
!pip install torchvision

In [7]:
from simpletransformers.ner import NERModel,NERArgs

In [8]:
model_dir_name = "custom_entity_extractor_bert"
model_dir_parent = "/Users/ash/Desktop"
output_dir = os.path.join(model_dir_parent, model_dir_name)
os.makedirs(output_dir, exist_ok=True)
new_model_name = "custom_entity_extraction_bert_model"
print("created", output_dir)

created /Users/ash/Desktop/custom_entity_extractor_bert1


In [9]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [10]:
print("starting time: " + time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(time.time())))

model = NERModel('bert', 'bert-base-cased', labels = ['O', 'BASE'], args = args, use_cuda = False)
model.train_model(train_data.loc[:, train_data.columns != 'text'], eval_data = test_data.loc[:, test_data.columns != 'text'], acc = accuracy_score)
result, model_outputs, preds_list = model.eval_model(test_data)

# Save model 
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    model_name = os.path.join(output_dir, "model.pkl")
    pickle.dump(model, open(model_name, 'wb'))
    print("Saved model to", output_dir)

print("ending time: " + time.strftime('%m/%d/%Y %H:%M:%S', time.localtime(time.time())))

starting time: 03/05/2022 12:23:42


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

  0%|          | 0/1 [00:00<?, ?it/s]

/Users/ash/opt/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

Saved model to /Users/ash/Desktop/custom_entity_extractor_bert1
ending time: 03/05/2022 14:02:16


/Users/ash/opt/anaconda3/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: BASE seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [11]:
result

{'eval_loss': 0.5288993914922079,
 'precision': 0.8596491228070176,
 'recall': 0.8882175226586103,
 'f1_score': 0.87369985141159}

In [12]:
def del_from_list(del_arr, main_arr):
    for x in del_arr:
        if x in main_arr:
            main_arr.remove(x)
    return main_arr

In [13]:
def calc_res(user_ans, answers):    
    final_res = []
    grade_ent_res = {}
    tmp_del = []
    user = "machine"

    true_positive = 0
    false_negative = 0
    false_positive = 0
    complete = 0
    ent = "base"
    grade_ent_res[user] = {}
    if ent not in grade_ent_res[user]:
        grade_ent_res[user][ent] = {"tp": 0, "fp": 0, "fn": 0, "complete": 0}

    if answers == user_ans:
        # 1. when both arrays are exactly the same
        for ans in user_ans:
            tmp_del.append(ans)
            true_positive += 1
            complete += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []
    else:
        # 2. when some elements are equal in both user and actual ans
        for ans in user_ans:
            if ans in answers:
                tmp_del.append(ans)
                true_positive += 1
                complete += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # 3. when user ans is partially correct
        # - if user ans is contained in actual ans
        if user_ans:
            for ans in user_ans:
                substring_user = [string for string in answers if ans in string]
                if substring_user:
                    tmp_del.append(ans)
                    true_positive += 1
                    false_positive += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # - if actual ans is contained in user ans
        if answers:
            for ans in answers:
                substring_ans = [string for string in user_ans if ans in string]
                if substring_ans:
                    tmp_del.append(ans)
                    true_positive += 1
                    false_positive += 1
        answers = del_from_list(tmp_del, answers)
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

        # 4. when actual ans is missing from user ans
        if answers and user_ans:
            set_difference_actual = set(answers) - set(user_ans)
            for ans in list(set_difference_actual):
                tmp_del.append(ans)
                false_negative += 1
        answers = del_from_list(tmp_del, answers)
        tmp_del = []

        # 5. when user ans is missing from actual ans
        if answers and user_ans:
            set_difference_user = set(user_ans) - set(answers)
            for ans in list(set_difference_user):
                tmp_del.append(ans)
                false_positive += 1
        user_ans = del_from_list(tmp_del, user_ans)
        tmp_del = []

    #print("tp: {}, fp: {}, fn: {}".format(true_positive, false_positive, false_negative))
    grade_ent_res[user][ent]["tp"] += true_positive
    grade_ent_res[user][ent]["fp"] += false_positive
    grade_ent_res[user][ent]["fn"] += false_negative
    grade_ent_res[user][ent]["complete"] += complete

    return grade_ent_res

In [14]:
result = os.path.join(output_dir, "bert_result.csv")
all_bases = 0
true_pos = 0
correct_bases = 0
false_pos = 0
false_neg = 0
final_res = []
headers = ["text", "predicted_base", "actual_base"]
final_res.append(headers)

for s in test_data["text"].unique():
    tmp_ans = []
    prediction, model_output = model.predict([s])
    actual_ans = ans_dict[s]
    for x in prediction[0]:
        for key, val in x.items():
            if val == "BASE":
                tmp_ans.append(key)
    all_bases += len(actual_ans)
    res_dict = calc_res(tmp_ans, actual_ans)
    true_pos += res_dict["machine"]['base']['tp']
    false_pos += res_dict["machine"]['base']['fp']
    false_neg += res_dict["machine"]['base']['fn']
    correct_bases += res_dict["machine"]['base']['complete']
    final_res.append([s, tmp_ans, actual_ans])

acc = ((correct_bases*100)/all_bases)
print("False pos: ", false_pos)
print("False neg: ", false_neg)
print("Marked {} out of {} bases correctly. Accuracy: {}".format(correct_bases, all_bases, acc))
    
with open(result, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(final_res)

print("created result file", result)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

{'machine': {'base': {'tp': 8, 'fp': 6, 'fn': 2, 'complete': 2}}}


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

{'machine': {'base': {'tp': 3, 'fp': 1, 'fn': 0, 'complete': 2}}}


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

{'machine': {'base': {'tp': 4, 'fp': 3, 'fn': 1, 'complete': 1}}}
False pos:  121
False neg:  73
Marked 214 out of 331 bases correctly. Accuracy: 64.65256797583082
created result file /Users/ash/Desktop/custom_entity_extractor_bert1/result.csv
